In [4]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import pandas as pd
df2 = pd.read_csv('LBMA-GOLD.csv',header=0,index_col=0,parse_dates = True)
df1 = pd.read_csv('BCHAIN-MKPRU.csv',header=0, index_col=0,parse_dates=True)
df1.reset_index().drop('Date',axis=1,inplace=True)
df1.head()

,Value
Date,
2016-09-11,621.65
2016-09-12,609.67
2016-09-13,610.92
2016-09-14,608.82
2016-09-15,610.38


In [6]:
data_train =df1.iloc[:int(df1.shape[0] * 0.7), :]
data_test = df1.iloc[int(df1.shape[0] * 0.7):, :]
print(data_train.shape, data_test.shape)

(1278, 1) (548, 1)


In [7]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler
import time
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler.fit(data_train)

MinMaxScaler(feature_range=(-1, 1))

In [8]:
data_train = scaler.transform(data_train)
data_test = scaler.transform(data_test)
data_train

array([[-0.99708325],
       [-0.99835067],
       [-0.99821842],
       ...,
       [-0.21232941],
       [-0.22369596],
       [-0.22861328]])

In [10]:
from keras.layers import Input, Dense, LSTM
from keras.models import Model

from keras.layers import *
from keras.models import *
# from keras.optimizers import Adam
from keras.optimizers import adam_v2

adam = adam_v2.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

output_dim = 1
batch_size = 256 #每轮训练模型时，样本的数量
epochs = 60 #训练60轮次
seq_len = 5
hidden_size = 128


TIME_STEPS = 5
INPUT_DIM = 1

lstm_units = 64
X_train = np.array([data_train[i : i + seq_len, :] for i in range(data_train.shape[0] - seq_len)])
y_train = np.array([data_train[i + seq_len, 0] for i in range(data_train.shape[0]- seq_len)])
X_test = np.array([data_test[i : i + seq_len, :] for i in range(data_test.shape[0]- seq_len)])
y_test = np.array([data_test[i + seq_len, 0] for i in range(data_test.shape[0] - seq_len)])

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)


(1273, 5, 1) (1273,) (543, 5, 1) (543,)


D:\anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [11]:
inputs = Input(shape=(TIME_STEPS, INPUT_DIM))
#drop1 = Dropout(0.3)(inputs)

x = Conv1D(filters = 64, kernel_size = 1, activation = 'relu')(inputs)  #, padding = 'same'
#x = Conv1D(filters=128, kernel_size=5, activation='relu')(output1)#embedded_sequences
x = MaxPooling1D(pool_size = 5)(x)
x = Dropout(0.2)(x)
print(x.shape)

(None, 1, 64)


In [12]:
lstm_out = Bidirectional(LSTM(lstm_units, activation='relu'), name='bilstm')(x)
#lstm_out = LSTM(lstm_units,activation='relu')(x)
print(lstm_out.shape)

(None, 128)


In [14]:
from keras import backend as K
# from keras.engine.topology import Layer
from tensorflow.keras.layers import Layer, InputSpec
import numpy as np
from keras import initializers
# Attention GRU network  未用     
class AttLayer(Layer):
    def __init__(self, **kwargs):
        self.init = initializers.get('normal')
        #self.input_spec = [InputSpec(ndim=3)]
        super(AttLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape)==128
        #self.W = self.init((input_shape[-1],1))
        self.W = self.init((input_shape[-1],))
        #self.input_spec = [InputSpec(shape=input_shape)]
        self.trainable_weights = [self.W]
        super(AttLayer, self).build(input_shape)  # be sure you call this somewhere!

    def call(self, x, mask=None):
        eij = K.tanh(K.dot(x, self.W))
        
        ai = K.exp(eij)
        weights = ai/K.sum(ai, axis=1).dimshuffle(0,'x')
        
        weighted_input = x*weights.dimshuffle(0,1,'x')
        return weighted_input.sum(axis=1)

    def get_output_shape_for(self, input_shape):
        return (input_shape[0], input_shape[-1])

In [16]:
from keras.layers import Input, Dense, merge
from keras import layers
# ATTENTION PART STARTS HERE
attention_probs = Dense(128, activation='sigmoid', name='attention_vec')(lstm_out)
#attention_mul=layers.merge([stm_out,attention_probs], output_shape],mode='concat',concat_axis=1))
attention_mul =Multiply()([lstm_out, attention_probs])
#attention_mul = merge([lstm_out, attention_probs],output_shape=32, name='attention_mul', mode='mul')

In [17]:
output = Dense(1, activation='sigmoid')(attention_mul)
#output = Dense(10, activation='sigmoid')(drop2)

model = Model(inputs=inputs, outputs=output)
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 5, 1)]       0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 5, 64)        128         ['input_1[0][0]']                
                                                                                                  
 max_pooling1d (MaxPooling1D)   (None, 1, 64)        0           ['conv1d[0][0]']                 
                                                                                                  
 dropout (Dropout)              (None, 1, 64)        0           ['max_pooling1d[0][0]']          
                                                                                              

In [ ]:
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, shuffle=False)
y_pred = model.predict(X_test)
print('MSE Train loss:', model.evaluate(X_train, y_train, batch_size=batch_size))
print('MSE Test loss:', model.evaluate(X_test, y_test, batch_size=batch_size))
plt.plot(y_test, label='test')
plt.plot(y_pred, label='pred')
plt.legend()
plt.show()

Epoch 1/60


In [ ]:
from sklearn.model_selection import train_test_split
df = pd.read_csv("./pollution.csv")
df = df.drop(['date','wnd_dir'], axis = 1)
train_size = int(len(df)*0.8)
train = df.iloc[:train_size,:]
test = df.iloc[train_size:,:]
#train, test = train_test_split(df, test_size=0.1)
print("len(train):",len(train))
print("len(test):",len(test))